# Genome Detective output conversion to CAMI profiling

Author: Sam Nooij  
Date: 2018-05-24


Input: ## what exactly do I need...?
- sample ID
- taxon
- number of reads
- total reads

    - These data (for both the assignments and the discoveries) can be derived from: the "bokeh input", output by `GenomeDetective_heatmaps.py`

- NCBI taxonomy DB (through ETE toolkit, see [this tutorial](http://etetoolkit.org/docs/2.3/tutorial/tutorial_ncbitaxonomy.html))

Output: table in [CAMI profiling format](https://github.com/bioboxes/rfc/blob/60263f34c57bc4137deeceec4c68a7f9f810f6a5/data-format/profiling.mkd)

Required python packages:
 - pandas
 - ete3
 
For automatic use in snakemake. The corresponding snakemake rule should provide the input:
 - the parsed report file ("results/GenomeDetective_results.csv")
 - a name for the output (e.g. "results/GenomeDetective_CAMI-profiling.tsv")
 
  ** Remember that an output has to be generated for each sample, separately! **

In [1]:
#Import required python libraries
import pandas as pd         #dataframe and csv export
from ete3 import NCBITaxa   #work with NCBI taxonomy

#DATA_TABLE = snakemake.input[0]
#PROFILES = snakemake.output[0]

In [2]:
# debugging cell:
DATA_TABLE = "../tmp/bokeh_input.csv"
PROFILE = "../results/GenomeDetective_CAMI-profiling.tsv"

In [136]:
results_df = pd.read_csv(DATA_TABLE)
samples = sorted(set(results_df["sample"]))

set_taxonomy = "superkingdom|Baltimore|phylum|class|order|family|genus|species"
taxonomy_order = set_taxonomy.split('|')
baltimore_taxa = {35237 : "Baltimore", #dsDNA, no RNA stage (class I)
                  2080735 : "Baltimore", #DNA viruses (separate group made for an Archaeal virus, should be class I)
                  29258 : "Baltimore", #ssDNA viruses (class II)
                  35325 : "Baltimore", #dsRNA viruses (class III)
                  35278 : "Baltimore", #ssRNA positive-strand viruses (class IV)
                  35301 : "Baltimore", #ssRNA negative-strand viruses (class V)
                  35268 : "Baltimore", #Retro-transcribing viruses, dsDNA-RT viruses (class VII)                             
                  }

#Retroviruses have no annotated Baltimore class in NCBI Taxonomy
#if order == 2169561 or order == "Ortervirales":
#    baltimore = "ssRNA-RT viruses" #(class VI)

"""

    I: dsDNA viruses (e.g. Adenoviruses, Herpesviruses, Poxviruses)
    II: ssDNA viruses (+ strand or "sense") DNA (e.g. Parvoviruses)
    III: dsRNA viruses (e.g. Reoviruses)
    IV: (+)ssRNA viruses (+ strand or sense) RNA (e.g. Picornaviruses, Togaviruses)
    V: (−)ssRNA viruses (− strand or antisense) RNA (e.g. Orthomyxoviruses, Rhabdoviruses)
    VI: ssRNA-RT viruses (+ strand or sense) RNA with DNA intermediate in life-cycle (e.g. Retroviruses)
    VII: dsDNA-RT viruses DNA with RNA intermediate in life-cycle (e.g. Hepadnaviruses)

source: https://en.wikipedia.org/wiki/Baltimore_classification

"""

PROFILES = [ "%s/%s_%s" % ('/'.join(PROFILE.split('/')[:-1]), sample, PROFILE.split('/')[-1]) for sample in samples ]

ncbi = NCBITaxa()
#In case you need to update, uncomment the next line:
#ncbi.update_taxonomy_database()

#Create separate files for each sample
for sample in samples:
    #Pick the right (pro)file name, that matches the sample
    profile = [ p for p in PROFILES if sample in p ][0]
    
    subset = results_df[results_df["sample"] == sample]
    taxa = subset["Assignment"]
    total_percentages = subset["percentage_of_total_reads"]
    
    rank_list_list = []
    output_list = []
    taxid_list = []
    ranks = []
    taxpath_list = []
    taxpathsn_list = []
    percentage_list = []
    
    for name in taxa:
        #remove names that have some addition in brackets,
        # like " (segment 1)"
        if ' (' in name:
            original_name = name
            name = name[:name.index(' (')]
        else:
            pass
        
        taxon_and_id = ncbi.get_name_translator([name])
        #ncbi.get_name_translator() returns a dictionary { 'taxon' : [id]}
        taxid = taxon_and_id[name]
        #taxid is a list with one number
        taxid_nr = taxid[0]
        taxid_list.append(taxid_nr)

        rank_dict = ncbi.get_rank(taxid)
        #ncbi.get_rank() requires a list of IDs, and returns a dictionary:
        # {id: 'rank'}
        rank = rank_dict[taxid_nr]
        ranks.append(rank)

        tax_path_dict = ncbi.get_lineage_translator(taxid)#[taxid_nr]
        #ncbi.get_lineage_translator() requires a list of IDs, and returns
        # a dictionary {leaf_id: [root_id, node_id, leaf_id]}
        tax_path = tax_path_dict[taxid_nr][1:]

        tax_path_sn = []
        #with a for-loop you can translate the taxids in the list
        # 'tax_path' to their corresponding scientific names (sn)
        for t in tax_path:
            tax_path_sn.append(ncbi.get_taxid_translator([t])[t])

        rank_list = []
        #Making this list requires using a for-loop;
        # using the function on a list makes an UNORDERED dictionary
        #Also, since the path differs between branches, I will look
        # for the longest using a list of lists
        for taxid in tax_path:
            rank_dict = ncbi.get_rank([taxid])
            rank = rank_dict[taxid]
            rank_list.append(rank)

        rank_list_list.append(rank_list)    

        tax_path_string = '|'.join(map(str, tax_path))
        tax_path_sn_string = '|'.join(tax_path_sn)
        
        taxpath_list.append(tax_path_string)
        taxpathsn_list.append(tax_path_sn_string)
        
        try:
            percentage = subset.loc[subset["Assignment"] == name]["percentage_of_total_reads"].values[0]
        except:
            percentage = subset.loc[subset["Assignment"] == original_name]["percentage_of_total_reads"].values[0]

        percentage_list.append(percentage)
            
        output_line = "%s\t%s\t%s\t%s\t%s" % (taxid_nr, rank, tax_path_string, tax_path_sn_string, percentage)
        
        output_list.append(output_line)
        output_dict = { "TAXID" : taxid_list, "RANK" : ranks, "TAXPATH" : taxpath_list,
                      "TAXPATHSN" : taxpathsn_list, "PERCENTAGE" : percentage_list}
        
    longest_taxonomy = '|'.join(max(rank_list_list, key = len))
    
    #Read the specification for details about this header:
    #https://github.com/bioboxes/rfc/blob/60263f34c57bc4137deeceec4c68a7f9f810f6a5/data-format/profiling.mkd
    header = """# Taxonomic Profiling Output
@SampleID:%s
@Version:0.9.3
@Ranks:%s\t#the longest path in this sample: virus taxonomy is messy
@TaxonomyID:ncbi-taxonomy_2018-05-25
@@TAXID\tRANK\tTAXPATH\tTAXPATHSN\tPERCENTAGE
""" % (sample, longest_taxonomy)
    
    #with open(profile, 'w') as output_table:
    #    output_table.write(header)
    #    output_table.write('\n'.join(output_list))
    output_df = pd.DataFrame(output_dict)
    print(output_df)


      TAXID     RANK                                        TAXPATH  \
0    142786    genus                10239|439488|35278|11974|142786   
1   1211417  species                     10239|12333|156614|1211417   
2     36343  species           10239|35237|28883|10699|186532|36343   
3   1623305    genus                10239|35237|28883|10699|1623305   
4   1105171  species         10239|35237|28883|10699|196894|1105171   
5     31537  species           10239|35237|28883|10699|186532|31537   
6   1327037  species         10239|35237|28883|10699|196894|1327037   
7   1655644  species               10239|29258|10841|117574|1655644   
8   1176422  species         10239|35237|28883|10699|196894|1176422   
9   1921554  species        10239|35237|28883|10699|1921553|1921554   
10  1921555  species        10239|35237|28883|10699|1921553|1921555   
11  1734382  species         10239|35237|28883|10744|196895|1734382   
12  1676182  species               10239|29258|10841|117574|1676182   
13   1

In [137]:
all_taxa = set('|'.join(output_df["TAXPATH"]).split('|'))
#Take all rows in column "TAXPATH", join them together with '|',
# split them into a list and take the uniques with 'set'

rows = output_df.shape[0]
new_row = rows

for t in all_taxa:
    percentages = output_df[0:rows][output_df["TAXPATH"].str.contains(t)]["PERCENTAGE"]
    perc_sum = sum(percentages)
    rank = ncbi.get_rank([t])[int(t)]
    tax_path = ncbi.get_lineage_translator([t])[int(t)][1:]
    tax_path_sn = []
    #with a for-loop you can translate the taxids in the list
    # 'tax_path' to their corresponding scientific names (sn)
    for tax in tax_path:
        tax_path_sn.append(ncbi.get_taxid_translator([tax])[tax])
        
    if rank == "no rank":
        if int(t) in baltimore_taxa:
            rank = baltimore_taxa[int(t)]
        
    new_row += 1
    if rank in taxonomy_order:
        output_df.loc[new_row] = [int(t), rank, '|'.join(map(str, tax_path)), '|'.join(tax_path_sn), perc_sum]
    else:
        pass

### SORTER THANKS TO: https://stackoverflow.com/a/23483221
taxonomy_sorter_index = dict(zip(taxonomy_order, range(len(taxonomy_order))))
print(taxonomy_sorter_index)

output_df['taxonomy_sorter'] = output_df["RANK"].map(taxonomy_sorter_index)

output_df.sort_values(by = ["taxonomy_sorter", "PERCENTAGE"], ascending = [True, False], inplace = True)

output_df.drop('taxonomy_sorter', 1, inplace = True)
###

output_df.drop_duplicates(keep = "first", inplace = True)

output_df

#Test: find and summarise all Picornaviridae:
#picorna_percentages = output_df[output_df["TAXPATH"].str.contains('12058')]["PERCENTAGE"]
#picorna_sum = sum(picorna_percentages)
#picorna_rank = ncbi.get_rank([12058])[12058]

#print(picorna_percentages, "\n", round(picorna_sum, 3), picorna_rank)

{'superkingdom': 0, 'Baltimore': 1, 'phylum': 2, 'class': 3, 'order': 4, 'family': 5, 'genus': 6, 'species': 7}


/software/python/anaconda2/envs/genomedetective/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,TAXID,RANK,TAXPATH,TAXPATHSN,PERCENTAGE
14,10239,superkingdom,10239,Viruses,77.908180
24,35278,Baltimore,10239|439488|35278,Viruses|ssRNA viruses|ssRNA positive-strand vi...,77.775231
35,35237,Baltimore,10239|35237,"Viruses|dsDNA viruses, no RNA stage",0.132949
23,464095,order,10239|439488|35278|464095,Viruses|ssRNA viruses|ssRNA positive-strand vi...,64.971769
39,28883,order,10239|35237|28883,"Viruses|dsDNA viruses, no RNA stage|Caudovirales",0.132949
28,12058,family,10239|439488|35278|464095|12058,Viruses|ssRNA viruses|ssRNA positive-strand vi...,64.971769
22,11974,family,10239|439488|35278|11974,Viruses|ssRNA viruses|ssRNA positive-strand vi...,12.803462
25,10744,family,10239|35237|28883|10744,"Viruses|dsDNA viruses, no RNA stage|Caudoviral...",0.117288
27,10662,family,10239|35237|28883|10662,"Viruses|dsDNA viruses, no RNA stage|Caudoviral...",0.015662
15,12059,genus,10239|439488|35278|464095|12058|12059,Viruses|ssRNA viruses|ssRNA positive-strand vi...,60.296814


In [139]:
output_df.to_csv("../tmp/20180622_Test_new_profile.csv", index=False)